In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import cv2 as cv
from tqdm import tqdm
import os
import json

from lac.utils.plotting import plot_path_3d, plot_3d_points, plot_poses
from lac.utils.frames import invert_transform_mat
from lac.util import (
    rmse,
    get_positions_from_poses,
    positions_rmse_from_poses,
    rotations_rmse_from_poses,
    load_data,
)

%load_ext autoreload
%autoreload 2

In [ ]:
import symforce

try:
    symforce.set_epsilon_to_symbol()
except symforce.AlreadyUsedEpsilon:
    print("Already set symforce epsilon")
    pass
import symforce.symbolic as sf

from lac.localization.symforce_util import odometry_lander_relpose_fgo
from lac.localization.imu_recovery import (
    recover_rotation,
    recover_rotation_exact,
    recover_translation,
)
from lac.localization.fgo import FactorGraph

# Symforce testing


In [ ]:
import copy

p = sf.Pose3()
sf.Pose3(R=p.R, t=p.t)

# Load data


In [ ]:
# data_path = "../../output/imu_20hz/" + "data_log.json"
data_path = "../../output/LocalizationAgent_spiral_norocks"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)

## Extract IMU Odometry


In [ ]:
from lac.localization.imu_recovery import ImuEstimator

imu_estimator = ImuEstimator(initial_pose)
imu_recovery_poses = [initial_pose]
imu_recovery_deltas = []
gt_pose_deltas = []

for i in tqdm(range(len(imu_data))):
    imu_estimator.update(imu_data[i], exact=False)
    imu_recovery_poses.append(imu_estimator.get_pose())
    imu_recovery_deltas.append(imu_estimator.get_pose_delta())
    gt_pose_deltas.append(poses[i + 1] @ invert_transform_mat(poses[i]))

# TODO: replace this with more principled delta estimation (integration)

In [ ]:
N = -1
fig = go.Figure()
fig = plot_poses(poses[:N], fig=fig, no_axes=True, color="black", name="Ground truth")
fig = plot_poses(imu_recovery_poses[:N], fig=fig, no_axes=True, color="blue", name="IMU recovery")
fig.update_layout(height=700, width=1200, scene_aspectmode="data")
fig.show()

## Get fiducial measurements


In [ ]:
from lac.perception.vision import FiducialLocalizer

In [ ]:
fid_localizer = FiducialLocalizer()
cameras = ["FrontLeft", "Right"]

In [ ]:
cam = "Right"

fiducial_pose_measurements = {}

for cam in cameras:
    cam_images_path = os.path.join(data_path, cam)
    for img_path in tqdm(os.listdir(cam_images_path)):
        i = int(img_path.split(".")[0])
        img_gray = cv.imread(os.path.join(cam_images_path, img_path), cv.IMREAD_GRAYSCALE)
        pose_measurements = fid_localizer.estimate_rover_pose(img_gray, cam, lander_pose)
        if pose_measurements:
            fiducial_pose_measurements[i] = pose_measurements

In [ ]:
unordered_fiducial_pose_measurements = []

for measurements in fiducial_pose_measurements.values():
    unordered_fiducial_pose_measurements.extend(measurements)

unordered_fiducial_position_measurements = get_positions_from_poses(
    unordered_fiducial_pose_measurements
)

# FGO


In [ ]:
ODOM_R_SIGMA = 0.001  # for rotations
ODOM_T_SIGMA = 0.001  # for translations [m]
ODOM_SIGMA = np.ones(6)
ODOM_SIGMA[:3] *= ODOM_R_SIGMA
ODOM_SIGMA[3:] *= ODOM_T_SIGMA

FIDUCIAL_SIGMA = 0.01 * np.ones(6)

In [ ]:
factor_graph = FactorGraph(ODOM_SIGMA, FIDUCIAL_SIGMA)

Sliding window optimization


In [ ]:
N = len(poses)
UPDATE_RATE = 10
WINDOW_SIZE = 10

curr_pose = initial_pose
factor_graph.add_pose(0, initial_pose)

# i is step which is 0 for initial and starts at 1 for the first run_step call
for i in tqdm(range(1, N)):
    curr_pose = curr_pose @ imu_recovery_deltas[i - 1]
    factor_graph.add_pose(i, curr_pose)
    # factor_graph.add_odometry_factor(i, imu_recovery_deltas[i - 1])
    if i - 1 in fiducial_pose_measurements:
        # for meas in fiducial_pose_measurements[i]:
        #     factor_graph.add_pose_measurement_factor(i, meas)
        factor_graph.add_pose_measurement_factor(i, fiducial_pose_measurements[i - 1][0])

    if i % UPDATE_RATE == 0:
        result = factor_graph.optimize(window=(i - UPDATE_RATE, i))
        curr_pose = factor_graph.get_pose(i)

Optimize the whole graph


In [ ]:
result = factor_graph.optimize(window=(0, len(poses) - 1))

In [ ]:
fgo_poses = factor_graph.get_all_poses()

In [ ]:
fig = go.Figure()
fig = plot_poses(poses[:N], fig=fig, no_axes=True, color="black", name="Ground truth")
fig = plot_poses(imu_recovery_poses[:N], fig=fig, no_axes=True, color="blue", name="IMU recovery")
fig = plot_poses(fgo_poses[:N], fig=fig, no_axes=True, color="green", name="FGO")
fig = plot_3d_points(
    unordered_fiducial_position_measurements[:N],
    fig=fig,
    color="orange",
    name="Fiducial measurements",
)
fig.update_layout(height=900, width=1600, scene_aspectmode="data")
fig.show()

In [ ]:
fig.write_html("fgo_result.html")

# Simulated lander measurements


In [ ]:
lander_position = lander_pose[:3, 3]

# Lander range and bearing measurements
lander_pose_measurements = []  # relative pose of lander in rover frame
lander_range_measurements = []
lander_los_measurements = []

for i in range(len(poses)):
    t_i = poses[i][:3, 3]
    delta_t = t_i - lander_position
    # TODO: add noise
    lander_range_measurements.append(np.linalg.norm(delta_t))
    lander_pose_measurements.append(invert_transform_mat(poses[i]) @ lander_pose)
    lander_los_measurements.append(-delta_t / np.linalg.norm(delta_t))

In [ ]:
N_WINDOW = 40
N_SHIFT = 10
N = 10000


def sliding_window_fgo():
    init_poses = imu_recovery_poses[:N_WINDOW]
    fgo_poses = [None] * N
    k_max = (N - N_WINDOW) // N_SHIFT

    for k in (pbar := tqdm(range(k_max))):
        window = slice(N_SHIFT * k, N_SHIFT * k + N_WINDOW)
        odometry = imu_recovery_deltas[window][:-1]
        # odometry = gt_odometry[window][:-1]
        lander_measurements = lander_pose_measurements[window]

        opt_poses, result = odometry_lander_relpose_fgo(
            init_poses,
            lander_pose,
            odometry,
            lander_measurements,
            ODOM_SIGMA,
            LANDER_RELPOSE_SIGMA,
            debug=False,
        )
        fgo_poses[N_SHIFT * k : N_SHIFT * (k + 1)] = opt_poses[:N_SHIFT]

        init_poses[:-N_SHIFT] = opt_poses[N_SHIFT:]
        if k != k_max - 1:
            pose = opt_poses[-1]
            for i in range(N_SHIFT):
                init_poses[-N_SHIFT + i] = pose @ imu_recovery_deltas[window][-1]
                pose = init_poses[-N_SHIFT + i]

    return fgo_poses

In [ ]:
fgo_poses = sliding_window_fgo()

In [ ]:
# all poses not None in fgo_poses
fgo_poses = [pose for pose in fgo_poses if pose is not None]
N = len(fgo_poses)

fig = go.Figure()
fig = plot_poses(poses[:N], fig=fig, no_axes=True, color="black", name="Ground truth")
fig = plot_poses(imu_recovery_poses[:N], fig=fig, no_axes=True, color="blue", name="IMU recovery")
fig = plot_poses(fgo_poses[:N], fig=fig, no_axes=True, color="green", name="FGO")
fig.update_layout(height=700, width=1200, scene_aspectmode="data")
fig.show()